# Canvas Group Example

> Examples of a workflow of creating canvas groups.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# import module
from CanvasGroupy.canvas import CanvasGroup

# Canvas Authentication

All the credentials should be stored at a json file in the following format.

If `API_KEY` and/or `course_id` was passed into the initializer, it will call the following methods to load the relevant information about your canvas and your canvas course.

In [ ]:
credentials_fp = "../credentials.json"
credentials_fp = "../../../credentials.json" #| hide_line

# instansiate a new Canvas Group Object 
# if your class size is large, it will take around 2 minutes to grab all student info.
API_URL = "https://canvas.ucsd.edu/"

# your canvas course id, found on the top of canvas url
course_id = ...
course_id = 45059 #| hide_line
cg = CanvasGroup(credentials_fp,
                 API_URL,
                 course_id=course_id,
                 verbosity=1)

# Create / Assign Group in One Call

In [ ]:
show_doc(CanvasGroup.assign_canvas_group)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/canvas.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### CanvasGroup.assign_canvas_group

>      CanvasGroup.assign_canvas_group (group_name:str,
>                                       group_members:[<class'str'>],
>                                       in_group_category:str)

Create new groups and assign group member into the class in the `self.group_category`

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| group_name | str | group name, display on canvas |
| group_members | [<class 'str'>] | list of group member's SIS Login |
| in_group_category | str | specify which group category the group belongs to |
| **Returns** | **(<class 'canvasapi.group.Group'>, [<class 'str'>])** | **list of unsuccessful join** |

In this way, we could directly create canvas group by specifying the group name, members info, and the group category directly.

In [ ]:
member1 = "email"
member1 = "grader-cogs118a-01" #| hide_line
group, unsuccessful = cg.assign_canvas_group(
    group_name="TestGroup000", 
    group_members=[member1], 
    in_group_category="Test"
)

assert len(unsuccessful)==0

In Group Set: Test,
Group TestGroup000 Created!


In [ ]:
#| hide
group.delete()

Group(_requester=<canvasapi.requester.Requester object>, is_public=False, id=122853, name=TestGroup000, created_at=2023-04-27T23:28:55Z, created_at_date=2023-04-27 23:28:55+00:00, max_membership=None, join_level=invitation_only, group_category_id=16281, description=None, members_count=1, storage_quota_mb=1024, storage_quota_mb_date=1024-01-01 00:00:00+00:00, context_type=Course, course_id=45059, avatar_url=None, role=None, leader=None, has_submission=False, concluded=False)

Note that it will throw a Key Error if the target `in_group_category` did not exist in the course.

In [ ]:
cg.assign_canvas_group(
    group_name="Failed",
    group_members=[member1],
    in_group_category="Group Project 2"
)

KeyError: 'Group Project 2 did not found in the group categories. Try to create one with CanvasGroup.create_group_category'

## Fetch Students' GitHub username

In [ ]:
show_doc(CanvasGroup.fetch_username_from_quiz)

---

[source](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/CanvasGroupy/canvas.py#L150){target="_blank" style="float:right; font-size:smaller"}

### CanvasGroup.fetch_username_from_quiz

>      CanvasGroup.fetch_username_from_quiz (quiz_id:int,
>                                            csv_name='github_id.csv',
>                                            col_index=7)

Fetch the GitHub user name from the canvas quiz

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| quiz_id | int |  | quiz id of the username quiz |
| csv_name | str | github_id.csv | csv output name. |
| col_index | int | 7 | canvas quiz generated csv's question field column index |
| **Returns** | **dict** |  | **{SIS Login ID: github} dictionary** |

An example of the quiz question looks like the followings

![quiz_screenshot](imgs/quiz_example.png) 

On canvas, this is a _fill in the blank_ question with the following raw format: 

![quiz_raw](imgs/raw_question.png)

In [ ]:
quiz_id = 139061
github_ids = cg.fetch_username_from_quiz(
    quiz_id=quiz_id
)

Quiz: GitHub ID fetch! 
Generating Student Analaysis...
[====================] 100%
Report Generated!
The Question asked is 1389031: What is your GitHub ID? Absolutely No Typo Please.. 
Make sure this is the correct question where you asked student for their GitHub id.
If you need to change the index of columns, change the col_index argument of this call.


In [ ]:
show_doc(CanvasGroup.check_github_usernames)

---

### CanvasGroup.check_github_usernames

>      CanvasGroup.check_github_usernames (github_ids:dict, credentials_fp='')

batch check github username from student inputs.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| github_ids | dict |  | {email: github id} of student inputs, generated from self.fetch_username_from_quiz |
| credentials_fp | str |  | # credential file path. [Template of the credentials.json](https://github.com/FleischerResearchLab/CanvasGroupy/blob/main/nbs/credentials.json) |
| **Returns** | **dict** |  | **{email: github id} of unreasonable github id** |

In [ ]:
cg.check_github_usernames(github_ids, "../../../credentials.json")

User: 64987902 Not Found on GitHub
User: 122977555 Not Found on GitHub
User: 10285923 Not Found on GitHub


{'gng': '64987902', 'y3guo': '122977555', 'nyanekch': '10285923'}

Since some students did not put their github username correctly.